Загрузка библиотек и данных

In [1]:
!pip install simpletransformers==0.61.13
!pip uninstall transformers
!pip install transformers==4.10.0
!git clone https://github.com/GoldenRMT/WikiSearch.git
!pip install googledrivedownloader

     |████████████████████████████████| 221 kB 5.4 MB/s 
     |████████████████████████████████| 124 kB 33.2 MB/s 
     |████████████████████████████████| 1.7 MB 30.1 MB/s 
     |████████████████████████████████| 287 kB 43.6 MB/s 
     |████████████████████████████████| 8.3 MB 30.8 MB/s 
     |████████████████████████████████| 1.2 MB 39.2 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 3.3 MB 31.8 MB/s 
     |████████████████████████████████| 2.9 MB 35.3 MB/s 
     |████████████████████████████████| 596 kB 41.4 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 895 kB 39.0 MB/s 
     |████████████████████████████████| 139 kB 45.7 MB/s 
     |████████████████████████████████| 97 kB 6.6 MB/s 
     |████████████████████████████████| 180 kB 43.8 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 125 kB 43.1 MB/s 
     |█████████████████

Found existing installation: transformers 4.11.3
Uninstalling transformers-4.11.3:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.7/dist-packages/transformers-4.11.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/transformers/*
Proceed (y/n)? н
Your response ('н') was not one of the expected responses: y, n
Proceed (y/n)? y
  Successfully uninstalled transformers-4.11.3
     |████████████████████████████████| 2.8 MB 5.2 MB/s 
Cloning into 'WikiSearch'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 96 (delta 33), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (96/96), done.


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('wordnet')
stopwords = stopwords.words("english")
lemmatizer = nltk.stem.WordNetLemmatizer() 

from nltk.tokenize import RegexpTokenizer, word_tokenize, sent_tokenize
main_tokenizer = RegexpTokenizer(r'\w+',)
sec_tokenizer = RegexpTokenizer(r'\S+')
 
from sklearn.externals import joblib
import numpy as np
from google.colab import output
import urllib
import difflib
import WikiSearch.wikipedia.wikipedia as wikipedia
import pandas as pd
from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [3]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='13Nuwm7BV-4RXI9JqjPTDE9rcdupkKqlF',
                                    dest_path='/Data/AIIJC/aiijc_1578_goodFromTrain_pretrained.model')

Функции для предобработки текста

In [4]:
def normal_form(word): #Получение нормальной формы слова
  word = word.lower()
  return word
 
def clean_html(html): #Очистка html
  soup = BeautifulSoup(BeautifulSoup(html, "lxml").text)
  return str(soup.body)

def get_good_tokens(text): #Выделение ключевых слов
  good_tokens = []

  for tokens in tokenizer(text)[1]:
    for token in tokens:
      token = normal_form(token)
      if token not in stopwords:
        good_tokens.append(token)
  return good_tokens

def tokenizer(text): #Токенизация текста в обработанные и необработанные токены
  raw_tokens = sec_tokenizer.tokenize(text)
  clean_tokens = main_tokenizer.tokenize_sents(raw_tokens)
  
  nClean_tokens = []
  for i in range(len(clean_tokens)):
    nClean_tokens.append([])
    for m in range(len(clean_tokens[i])):
      if normal_form(clean_tokens[i][m]) != 's':
        nClean_tokens[i].append(normal_form(clean_tokens[i][m]))

  return (raw_tokens, nClean_tokens)
 
def similarity(s1, s2): #Нахождение коэффициента схожести между двумя строками
  normalized1 = s1.lower()
  normalized2 = s2.lower()
  matcher = difflib.SequenceMatcher(None, normalized1, normalized2)
  return matcher.ratio()

def part_extractor(data,question,step,part_length): #Функция выделения релевантного фрагмента (Текст, вопрос, длинна фрагмента)
  good_tokens = get_good_tokens(question)
  
  tokens = tokenizer(data)
  
    
  for i in range(step-(len(tokens[0]) % step)): #Увеличение количества токенов до кратного длины части
    tokens[0].append('')
    tokens[1].append('')
 

  match_counter = 0 #Счетчик точных совпадений токенов
  best_part = '' #Лучшая часть
  max_match_qty = 0 #Максимальное количество совпавших токенов
 
  main_clrTokens = tokens[1]
  main_tokens = tokens[0]
  
  for i in range(0,len(tokens[0])-1,part_length): #Нахождение наиболее релевантной части текста
    tokens = main_tokens[i:i+part_length-1]
    clrTokens = main_clrTokens[i:i+part_length-1]

    for good_token in good_tokens:
      if in_tokens(good_token,clrTokens):
        match_counter += 1

    if match_counter > max_match_qty:
      max_match_qty = match_counter
      best_part = tokens
      
   
    match_counter = 0
  
  

  fin = '' #Восстановление текста
  for i in best_part:
    fin += (i+' ')

  
  return fin

def in_tokens(token,text):
  for i in text:
    for m in i:
      if token == m:
        return True
  return False

Загрузка модели и функция для ответов на вопрос

In [5]:
model = joblib.load('/Data/AIIJC/aiijc_1578_goodFromTrain_pretrained.model')

model.args.max_seq_length = 512
model.args.silent = True

In [6]:
def answering(question):
  text = question

  good_tokens = get_good_tokens(text)
  
  try:
    urls = wikipedia.search(text,results=2)
  except:
    link_1 = '-'
    link_2 = '-'

  try:
    link_1 = urls[0]
  except:
    link_1 = '-'

  try:
    link_2 = urls[1]
  except:
    link_2 = '-'

  #Загрузка статей википедии
  try:
    link_1 = link_1.replace('https://en.wikipedia.org/wiki/','') #Убераем начало ссылки
    link_1 = urllib.parse.unquote(link_1) #Заменяем кривые символы на оригинал
    data_1 = wikipedia.page(link_1,auto_suggest=False).content #Парсим страничку вики
    data_1 = data_1.replace('\n',' ')
  except:
    pass
  try:
    link_2 = link_2.replace('https://en.wikipedia.org/wiki/','') #Убераем начало ссылки
    link_2 = urllib.parse.unquote(link_2) #Заменяем кривые символы на оригинал
    data_2 = wikipedia.page(link_2,auto_suggest=False).content #Парсим страничку вики
    data_2 = data_2.replace('\n',' ')
  except:
    pass
  

  try: #Поиск релевантного куска длиной 128 токенов с шагом 64 в самой релевантной статье
    context = part_extractor(data_1,question,16,64)
  except:
    pass
 
  try: #Поиск релевантного куска длиной 64 токена с шагом 32 во второй по релевантности статье
    context += ' ' + part_extractor(data_2,question,16,32)
  except:
    pass

  try:
    predict = model.predict([{'context': context,'qas': [{'id': 0, 'question': question}]}])[0] #Предсказание ответа
  except:
    predict = [{'answer':['']}]
    predict[0]['answer'][0] = 'empty'


  if predict[0]['answer'][0] == 'empty':
    try:
      context = part_extractor(data_1,question,16,64)
      predict = model.predict([{'context': context,'qas': [{'id': 0, 'question': question}]}])[0]
    except:
      pass

  if predict[0]['answer'][0] == 'empty':
    try:
      context = part_extractor(data_2,question,16,64)
      predict = model.predict([{'context': context,'qas': [{'id': 0, 'question': question}]}])[0]
    except:
      pass

  if predict[0]['answer'][0] == 'empty':
    try:
      context = part_extractor(data_1,question,16,128)
      predict = model.predict([{'context': context,'qas': [{'id': 0, 'question': question}]}])[0]
    except:
      pass
  
  if predict[0]['answer'][0] == 'empty':
    try:
      context = part_extractor(data_2,question,16,128)
      predict = model.predict([{'context': context,'qas': [{'id': 0, 'question': question}]}])[0]
    except:
      pass
 
  if predict[0]['answer'][0] == 'empty':
    try:
      context = part_extractor(data_1,question,16,256)
      predict = model.predict([{'context': context,'qas': [{'id': 0, 'question': question}]}])[0]
    except:
      pass
 
  if predict[0]['answer'][0] == 'empty':
    try:
      context = part_extractor(data_2,question,16,256)
      predict = model.predict([{'context': context,'qas': [{'id': 0, 'question': question}]}])[0]
    except:
      pass

  return predict[0]['answer'][0]

Проверка работоспособности и времени работы функции

In [8]:
import time

time_1 = time.time()

print(answering("What is the name of Trump first daughter?")) 

print('Время обработки запроса: ' + str(time.time()-time_1))

Ivana Marie "Ivanka" Trump
Время обработки запроса: 1.6687853336334229
